In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  


import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
          #   "mean_40", "variance_40",
          #   "mean_50", "variance_50",
             "cap", "normalization"]

rows = []

In [3]:
# 3 classes

NUM_CLASSES_REDUCED = 3
nums = (0, 3, 1)
ratio = (200, 20, 1)

norm=True

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[134.1855, 122.7346, 118.3749], std=[70.5125, 64.4848, 66.5604])])
else:
    transform=None

    
train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=transform)  

test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=transform)  

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)


reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)
targets = ratio_train_CIFAR10.labels 
class_count = np.unique(targets, return_counts=True)[1]

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)

triplet_train_CIFAR10 = class_sampling.ForTripletLoss(reduced_train_CIFAR10, smote=False, transform=transform, num_classes=3)
triplet_ratio_train_CIFAR10 = class_sampling.ForTripletLoss(ratio_train_CIFAR10, smote=False, transform=transform, num_classes=3)
triplet_smote_train_CIFAR10 = class_sampling.ForTripletLoss(smote_train_CIFAR10, smote=True, transform=transform, num_classes=3)

weight = 1. / class_count
samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= max(class_count)

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss = DataLoader(triplet_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_ratio = DataLoader(triplet_ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_smote = DataLoader(triplet_smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# 3 class normal

learning_rates = [1e-2, 5e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 3, nums, (1, 1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.001101137677828471, AUC: 0.48153950000000006


Test set: Avg. loss: 0.0008184388677279155, AUC: 0.8536186666666667


Test set: Avg. loss: 0.0006444128354390462, AUC: 0.9132271666666666


Test set: Avg. loss: 0.0006369576851526896, AUC: 0.9185373333333334


Test set: Avg. loss: 0.0011027837991714477, AUC: 0.4255000833333333


Test set: Avg. loss: 0.0007888296842575073, AUC: 0.8538959999999999


Test set: Avg. loss: 0.0006802848180135091, AUC: 0.8962965


Test set: Avg. loss: 0.0006800517241160075, AUC: 0.9124463333333334


Test set: Avg. loss: 0.0011040879090627035, AUC: 0.442987


Test set: Avg. loss: 0.0007678199013074239, AUC: 0.8738281666666667


Test set: Avg. loss: 0.0006366982062657674, AUC: 0.9103801666666667


Test set: Avg. loss: 0.0005771947900454203, AUC: 0.9220054166666666


Test set: Avg. loss: 0.0011034004290898642, AUC: 0.5220113333333333


Test set: Avg. loss: 0.0007123484214146932, AUC: 0.8829556666666667


Test set: Avg. loss: 0.000621326545874

In [7]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [8]:
#  3 class ratio

learning_rates =  [1e-2, 1e-3, 5e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.001102492650349935, AUC: 0.4337535833333333


Test set: Avg. loss: 0.0028033832708994548, AUC: 0.6646945


Test set: Avg. loss: 0.0025717381636301676, AUC: 0.6802568333333333


Test set: Avg. loss: 0.002387967030207316, AUC: 0.6892651666666666


Test set: Avg. loss: 0.0010961395899454752, AUC: 0.5539086666666666


Test set: Avg. loss: 0.0024249297777811686, AUC: 0.6684312499999999


Test set: Avg. loss: 0.002277838150660197, AUC: 0.6947530000000001


Test set: Avg. loss: 0.002308895746866862, AUC: 0.7068166666666666


Test set: Avg. loss: 0.0011040597756703694, AUC: 0.5133956666666667


Test set: Avg. loss: 0.002486731767654419, AUC: 0.668329


Test set: Avg. loss: 0.0023136560916900634, AUC: 0.6831764166666666


Test set: Avg. loss: 0.002032056490580241, AUC: 0.6958366666666667


Test set: Avg. loss: 0.00110069207350413, AUC: 0.5184295


Test set: Avg. loss: 0.0026118515332539875, AUC: 0.6661951666666667


Test set: Avg. loss: 0.0023768699169158937, AUC: 0.6801

In [9]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [10]:
# 3 class oversampled 

learning_rates = [1e-2, 1e-3, 5e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.0010989482005437216, AUC: 0.5574586666666667


Test set: Avg. loss: 0.0014568779468536377, AUC: 0.7987512499999999


Test set: Avg. loss: 0.0027695000171661377, AUC: 0.7861481666666666


Test set: Avg. loss: 0.0033176114559173584, AUC: 0.7812484166666667


Test set: Avg. loss: 0.0010912991762161255, AUC: 0.5963589166666666


Test set: Avg. loss: 0.0017322154839833578, AUC: 0.765361


Test set: Avg. loss: 0.0027109846274058023, AUC: 0.7597248333333334


Test set: Avg. loss: 0.0033343732357025146, AUC: 0.7680907499999999


Test set: Avg. loss: 0.0011008808612823487, AUC: 0.5308455


Test set: Avg. loss: 0.0016412200133005777, AUC: 0.7837315


Test set: Avg. loss: 0.0028054415384928385, AUC: 0.7703296666666667


Test set: Avg. loss: 0.0035819047292073567, AUC: 0.7849685000000001


Test set: Avg. loss: 0.0011016372044881184, AUC: 0.49567524999999996


Test set: Avg. loss: 0.0014501880804697672, AUC: 0.79916


Test set: Avg. loss: 0.0023616336981455485, AUC: 0.795649

In [11]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [12]:
#  3 class SMOTE

learning_rates = [1e-2, 1e-3, 5e-3]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = None

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.0011113398869832356, AUC: 0.45399366666666663


Test set: Avg. loss: 0.001039024035135905, AUC: 0.7600510833333334


Test set: Avg. loss: 0.0014980615377426147, AUC: 0.7576163333333333


Test set: Avg. loss: 0.0016282843351364137, AUC: 0.7855533333333332


Test set: Avg. loss: 0.0011053721904754638, AUC: 0.40628075


Test set: Avg. loss: 0.000999620020389557, AUC: 0.7641230833333333


Test set: Avg. loss: 0.00213276473681132, AUC: 0.7200933333333333


Test set: Avg. loss: 0.0018212167024612427, AUC: 0.7857006666666666


Test set: Avg. loss: 0.00110055140654246, AUC: 0.5410479166666667


Test set: Avg. loss: 0.00124763818581899, AUC: 0.7365446666666667


Test set: Avg. loss: 0.001190781593322754, AUC: 0.7589035833333334


Test set: Avg. loss: 0.0017052281697591146, AUC: 0.7550365


Test set: Avg. loss: 0.0010934751828511557, AUC: 0.582834


Test set: Avg. loss: 0.0010338213443756104, AUC: 0.7306260833333335


Test set: Avg. loss: 0.001256510615348816, AUC: 0.7601

In [13]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [14]:
# 3 class capped smote 


momentum=0
learning_rates = [1e-2, 1e-3, 5e-3]


cap_aucs = []

caps = [1, 5, 10]

for cap in caps:
    
    loss_fn_args = {}
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNet(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)


Test set: Avg. loss: 0.0011079707543055217, AUC: 0.4224414166666666


Test set: Avg. loss: 0.0024819238980611164, AUC: 0.6706555833333333


Test set: Avg. loss: 0.002305363098780314, AUC: 0.6838636666666668


Test set: Avg. loss: 0.002249549627304077, AUC: 0.6926545000000001


Test set: Avg. loss: 0.0011115634441375732, AUC: 0.44383383333333337


Test set: Avg. loss: 0.0018068215847015382, AUC: 0.7456852500000001


Test set: Avg. loss: 0.0023212509950002033, AUC: 0.7392154999999999


Test set: Avg. loss: 0.0015493378241856893, AUC: 0.8035191666666667


Test set: Avg. loss: 0.0010960235595703124, AUC: 0.5768070833333333


Test set: Avg. loss: 0.0017772521575291952, AUC: 0.7280496666666667


Test set: Avg. loss: 0.002030446648597717, AUC: 0.7317598333333333


Test set: Avg. loss: 0.002225544532140096, AUC: 0.7191306666666666


Test set: Avg. loss: 0.0011004793246587117, AUC: 0.5102983333333333


Test set: Avg. loss: 0.002655739943186442, AUC: 0.6715455


Test set: Avg. loss: 0.002424101


Test set: Avg. loss: 0.0008978073398272197, AUC: 0.7981495000000001


Test set: Avg. loss: 0.0010476662516593932, AUC: 0.797174


Test set: Avg. loss: 0.0013957345088322958, AUC: 0.801761


Test set: Avg. loss: 0.0011011792421340943, AUC: 0.5008129166666667


Test set: Avg. loss: 0.0010919816493988037, AUC: 0.7615899166666665


Test set: Avg. loss: 0.0017090630531311035, AUC: 0.75741025


Test set: Avg. loss: 0.0017677673896153768, AUC: 0.7701735833333333


Test set: Avg. loss: 0.0011115765571594238, AUC: 0.4346426666666667


Test set: Avg. loss: 0.0011828872362772624, AUC: 0.7415978333333332


Test set: Avg. loss: 0.0012650069793065388, AUC: 0.7911315833333333


Test set: Avg. loss: 0.0015466095209121704, AUC: 0.7957371666666666


Test set: Avg. loss: 0.001096881588300069, AUC: 0.5596155833333333


Test set: Avg. loss: 0.0011262359221776326, AUC: 0.7633


Test set: Avg. loss: 0.0018325774272282918, AUC: 0.7689121666666665


Test set: Avg. loss: 0.0017438907623291016, AUC: 0.797137833


Test set: Avg. loss: 0.0010901000102361044, AUC: 0.7558843333333334


Test set: Avg. loss: 0.001592508633931478, AUC: 0.7648861666666665


Test set: Avg. loss: 0.0015164965391159058, AUC: 0.793024


Test set: Avg. loss: 0.0011064399878184, AUC: 0.44293325


Test set: Avg. loss: 0.0011655902862548828, AUC: 0.7469121666666666


Test set: Avg. loss: 0.001499106486638387, AUC: 0.7580291666666666


Test set: Avg. loss: 0.0018441102107365926, AUC: 0.7539993333333332


Test set: Avg. loss: 0.0011045443614323934, AUC: 0.46125925000000007


Test set: Avg. loss: 0.0011498111089070639, AUC: 0.7564623333333333


Test set: Avg. loss: 0.0015798531770706177, AUC: 0.7602367499999999


Test set: Avg. loss: 0.002093439976374308, AUC: 0.7589875833333334


Test set: Avg. loss: 0.001103798508644104, AUC: 0.4563561666666667


Test set: Avg. loss: 0.0010767038265864055, AUC: 0.7555844999999999


Test set: Avg. loss: 0.0013137869834899901, AUC: 0.7909311666666666


Test set: Avg. loss: 0.0018448962370554607,

In [15]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [16]:
# 3 class euclidean distance capped smote 


momentum=0
learning_rates = [1e-2, 1e-3, 5e-3]


cap_aucs = []

caps = [1, 5, 10]

loss_fn_args = {}
loss_fn_args['distance'] = 'euclidean'


for cap in caps:
    
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["distance_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0011004855235417683, AUC: 0.4887625


Test set: Avg. loss: 0.0011042307217915854, AUC: 0.7546737499999999


Test set: Avg. loss: 0.001469256083170573, AUC: 0.7447474166666667


Test set: Avg. loss: 0.0018616555134455363, AUC: 0.7519853333333333


Test set: Avg. loss: 0.0011009955008824666, AUC: 0.4859675833333334


Test set: Avg. loss: 0.0014954797426859538, AUC: 0.7307878333333333


Test set: Avg. loss: 0.0014571794668833415, AUC: 0.7434764166666666


Test set: Avg. loss: 0.001918233076731364, AUC: 0.7510033333333332


Test set: Avg. loss: 0.0010990285078684488, AUC: 0.5017333333333333


Test set: Avg. loss: 0.0015441822210947673, AUC: 0.7289679166666666


Test set: Avg. loss: 0.0013621874650319417, AUC: 0.7503736666666666


Test set: Avg. loss: 0.0020592970053354897, AUC: 0.7446693333333333


Test set: Avg. loss: 0.001104023019472758, AUC: 0.4327043333333333


Test set: Avg. loss: 0.0011127008597056072, AUC: 0.7686645


Test set: Avg. loss: 0.00162692983945210


Test set: Avg. loss: 0.00109506889184316, AUC: 0.6075227500000001


Test set: Avg. loss: 0.0010930527051289877, AUC: 0.7969098333333333


Test set: Avg. loss: 0.00121285879611969, AUC: 0.7962791666666668


Test set: Avg. loss: 0.0015341360569000244, AUC: 0.7968086666666668


Test set: Avg. loss: 0.00110688320795695, AUC: 0.39512858333333334


Test set: Avg. loss: 0.0010997189680735271, AUC: 0.7778755


Test set: Avg. loss: 0.0013350268205006917, AUC: 0.77851625


Test set: Avg. loss: 0.0011274513800938924, AUC: 0.81341375


Test set: Avg. loss: 0.0011012434562047322, AUC: 0.49900916666666667


Test set: Avg. loss: 0.001257138729095459, AUC: 0.7524218333333333


Test set: Avg. loss: 0.002043050686518351, AUC: 0.7320953333333332


Test set: Avg. loss: 0.0026825573444366453, AUC: 0.7337874166666666


Test set: Avg. loss: 0.0010923410654067993, AUC: 0.6525029999999999


Test set: Avg. loss: 0.0009580798546473185, AUC: 0.7926976666666667


Test set: Avg. loss: 0.0015643794139226278, AUC: 0


Test set: Avg. loss: 0.0010996352036794026, AUC: 0.7698611666666667


Test set: Avg. loss: 0.0018259684642155966, AUC: 0.7553820833333332


Test set: Avg. loss: 0.001500830332438151, AUC: 0.7931090833333333


Test set: Avg. loss: 0.00109957222143809, AUC: 0.5198979166666666


Test set: Avg. loss: 0.001127657175064087, AUC: 0.75847275


Test set: Avg. loss: 0.0012819421291351317, AUC: 0.7650260000000001


Test set: Avg. loss: 0.0013352120717366536, AUC: 0.7699018333333333


Test set: Avg. loss: 0.0011067168315251669, AUC: 0.44954541666666664


Test set: Avg. loss: 0.0012927870750427245, AUC: 0.7444374166666666


Test set: Avg. loss: 0.0011801127195358277, AUC: 0.7940649999999999


Test set: Avg. loss: 0.0015621389547983806, AUC: 0.7898746666666666


Test set: Avg. loss: 0.0011187699635823568, AUC: 0.4121849166666667


Test set: Avg. loss: 0.0011955602169036864, AUC: 0.770215


Test set: Avg. loss: 0.0012250461181004843, AUC: 0.802267


Test set: Avg. loss: 0.0017819425264994303, AUC: 0

In [17]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [18]:
# 3 class cosine distance capped smote 


momentum=0
learning_rates = [1e-2, 1e-3, 5e-3]


cap_aucs = []

caps = [1, 5, 10]

loss_fn_args = {}
loss_fn_args['distance'] = 'cosine'


for cap in caps:
    
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0011052825848261514, AUC: 0.43978183333333337


Test set: Avg. loss: 0.001242885112762451, AUC: 0.7556419999999999


Test set: Avg. loss: 0.0017786058187484741, AUC: 0.7451270833333333


Test set: Avg. loss: 0.001355748494466146, AUC: 0.7903454166666667


Test set: Avg. loss: 0.0010999015967051188, AUC: 0.5015735


Test set: Avg. loss: 0.001183523694674174, AUC: 0.7529743333333333


Test set: Avg. loss: 0.0017083600759506227, AUC: 0.7684294999999999


Test set: Avg. loss: 0.0020539085070292156, AUC: 0.7834125


Test set: Avg. loss: 0.0010926770766576132, AUC: 0.5972151666666666


Test set: Avg. loss: 0.0012481575806935629, AUC: 0.7396061666666668


Test set: Avg. loss: 0.001280996561050415, AUC: 0.774436


Test set: Avg. loss: 0.0013489604791005452, AUC: 0.8045683333333334


Test set: Avg. loss: 0.0011085643370946248, AUC: 0.41532358333333336


Test set: Avg. loss: 0.001295187473297119, AUC: 0.7403531666666666


Test set: Avg. loss: 0.0014481489260991414, AUC: 0


Test set: Avg. loss: 0.001102006713549296, AUC: 0.5132036666666666


Test set: Avg. loss: 0.0012708915074666342, AUC: 0.7639296666666667


Test set: Avg. loss: 0.0015375802119572957, AUC: 0.7768043333333333


Test set: Avg. loss: 0.002539357344309489, AUC: 0.7442037499999999


Test set: Avg. loss: 0.0011142092148462932, AUC: 0.3598644166666667


Test set: Avg. loss: 0.0010708965063095093, AUC: 0.77946075


Test set: Avg. loss: 0.001908510684967041, AUC: 0.7509295000000001


Test set: Avg. loss: 0.0016529592672983805, AUC: 0.79660725


Test set: Avg. loss: 0.0011188224951426187, AUC: 0.3916071666666667


Test set: Avg. loss: 0.0012840594053268432, AUC: 0.7236514166666667


Test set: Avg. loss: 0.0012736543416976929, AUC: 0.7729545833333334


Test set: Avg. loss: 0.002117136081059774, AUC: 0.7873815000000001


Test set: Avg. loss: 0.0011116609573364258, AUC: 0.357033


Test set: Avg. loss: 0.0010108190377553304, AUC: 0.7509784166666668


Test set: Avg. loss: 0.0012108296553293863, AUC: 


Test set: Avg. loss: 0.001682835817337036, AUC: 0.7700859166666666


Test set: Avg. loss: 0.0016216527223587036, AUC: 0.8116925


Test set: Avg. loss: 0.0011027799050013225, AUC: 0.46017141666666667


Test set: Avg. loss: 0.0009993523557980854, AUC: 0.7918486666666666


Test set: Avg. loss: 0.0013525794744491577, AUC: 0.7881920833333332


Test set: Avg. loss: 0.0015977139075597127, AUC: 0.8012561666666667


Test set: Avg. loss: 0.001100353757540385, AUC: 0.47812058333333335


Test set: Avg. loss: 0.0011181455055872599, AUC: 0.7524395833333334


Test set: Avg. loss: 0.0015768436988194783, AUC: 0.7707788333333333


Test set: Avg. loss: 0.001684269905090332, AUC: 0.8039378333333334


Test set: Avg. loss: 0.0010977291266123454, AUC: 0.5273618333333334


Test set: Avg. loss: 0.0012737759749094646, AUC: 0.7475231666666667


Test set: Avg. loss: 0.001591060439745585, AUC: 0.7671234166666666


Test set: Avg. loss: 0.0019627333879470824, AUC: 0.7713725


Test set: Avg. loss: 0.0011134448846181

In [19]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [ ]:
# 3 class triplet loss capped smote


momentum=0
learning_rates = [1e-2, 1e-3, 5e-3]

cap_aucs = []

start_epoch = 2

loss_caps = [1, 5, 10]
loss_fn_args = {}


for loss_cap in loss_caps:
    
    loss_fn_args['loss_cap'] = loss_cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=loss_fns.TripletLoss,loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0011012449661890666, AUC: 0.4678569166666666


Test set: Avg. loss: 0.001351433793703715, AUC: 0.7307910833333334


Test set: Avg. loss: 0.001755292773246765, AUC: 0.732867


Test set: Avg. loss: 0.0018722213506698609, AUC: 0.7645645833333332


Test set: Avg. loss: 0.0011008409659067789, AUC: 0.48582624999999996


Test set: Avg. loss: 0.0012604947487513225, AUC: 0.7507433333333333


Test set: Avg. loss: 0.0017218114932378134, AUC: 0.7700464166666666


Test set: Avg. loss: 0.0022693723837534587, AUC: 0.7752346666666666


Test set: Avg. loss: 0.001106525460879008, AUC: 0.4166286666666667


Test set: Avg. loss: 0.0012869817018508912, AUC: 0.7328335


Test set: Avg. loss: 0.0014735660950342814, AUC: 0.7437383333333334


Test set: Avg. loss: 0.001680525263150533, AUC: 0.7754193333333333


Test set: Avg. loss: 0.001101671536763509, AUC: 0.48418674999999994


Test set: Avg. loss: 0.0011174571514129638, AUC: 0.7666405833333334


Test set: Avg. loss: 0.001347706437110900


Test set: Avg. loss: 0.0011168666283289592, AUC: 0.74792375


Test set: Avg. loss: 0.0015599504311879476, AUC: 0.7871136666666666


Test set: Avg. loss: 0.0018386873801549276, AUC: 0.8037405833333334


Test set: Avg. loss: 0.0011025656859079997, AUC: 0.4896275833333333


Test set: Avg. loss: 0.001231965978940328, AUC: 0.74530175


Test set: Avg. loss: 0.0016783792575200398, AUC: 0.7676095833333333


Test set: Avg. loss: 0.002151628653208415, AUC: 0.7773599999999999


Test set: Avg. loss: 0.0010982952117919921, AUC: 0.5357945833333333


Test set: Avg. loss: 0.0009739748438199361, AUC: 0.7773880833333333


Test set: Avg. loss: 0.0015675476392110188, AUC: 0.7916948333333332


Test set: Avg. loss: 0.0016550891796747844, AUC: 0.8078771666666666


Test set: Avg. loss: 0.00110202956199646, AUC: 0.4513276666666666


Test set: Avg. loss: 0.0010733749071756998, AUC: 0.7827265


Test set: Avg. loss: 0.001440328319867452, AUC: 0.7870779999999998


Test set: Avg. loss: 0.0017226599852244058, AUC: 

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 